In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

import os, json

with open("data/regelems_KG.json") as f:
    data = json.load(f)

G: nx.MultiDiGraph = json_graph.node_link_graph(data, edges="edges")
print(G)

MultiDiGraph with 6422 nodes and 669305 edges


In [2]:
def node_color(x):
    return "lightblue" if type(x) == str else "red"

In [3]:
# attempt visualization
#plt.figure(figsize=(12, 12))
#pos = nx.spring_layout(G)
#nx.draw(G, pos, with_labels=False, node_size=50, font_size=8, node_color=[node_color(n) for n in G.nodes()])
#plt.title("NCBI Functional Regulatory Elements Knowledge Graph")
#plt.show()

In [4]:
# centrality

in_degree = nx.in_degree_centrality(G)
in_sorted = sorted(in_degree.items(), key=lambda x: x[1], reverse=True)
print("Top 10 nodes by in-degree centrality:")
for node, centrality in in_sorted[:10]:
    print(f"Node: {node}, In-Degree Centrality: {centrality:.4f}")

out_degree = nx.out_degree_centrality(G)
out_sorted = sorted(out_degree.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 nodes by out-degree centrality:")
for node, centrality in out_sorted[:10]:
    print(f"Node: {node}, Out-Degree Centrality: {centrality:.4f}")

NDG = G.to_undirected()
betweenness = nx.betweenness_centrality(NDG)
betweenness_sorted = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 nodes by betweenness centrality:")
for node, centrality in betweenness_sorted[:10]:
    print(f"Node: {node}, Betweenness Centrality: {centrality:.4f}")

Top 10 nodes by in-degree centrality:
Node: 8161, In-Degree Centrality: 44.1043
Node: 7030, In-Degree Centrality: 16.6837
Node: 1422, In-Degree Centrality: 1.2032
Node: 3378, In-Degree Centrality: 1.0486
Node: 9414, In-Degree Centrality: 1.0477
Node: 4831, In-Degree Centrality: 0.9868
Node: 5984, In-Degree Centrality: 0.9541
Node: 7949, In-Degree Centrality: 0.9369
Node: 5741, In-Degree Centrality: 0.9047
Node: 9021, In-Degree Centrality: 0.7746

Top 10 nodes by out-degree centrality:
Node: <cls>, Out-Degree Centrality: 0.8478
Node: G, Out-Degree Centrality: 0.3898
Node: C, Out-Degree Centrality: 0.3755
Node: T, Out-Degree Centrality: 0.3398
Node: A, Out-Degree Centrality: 0.3269
Node: TTTTTT, Out-Degree Centrality: 0.1448
Node: AAAAAA, Out-Degree Centrality: 0.1420
Node: ATTTTT, Out-Degree Centrality: 0.0941
Node: AAAAAT, Out-Degree Centrality: 0.0927
Node: TATTTT, Out-Degree Centrality: 0.0911

Top 10 nodes by betweenness centrality:
Node: 8161, Betweenness Centrality: 0.1504
Node: 7

# Looking at f/8161 and f/7030 as Regulatory Element Indicators

Previous enrichment results seem to show f/8161 and f/7030 are strongly associated with regulatory functions, and intervention results suggest that f/8161 may also identify some sort of inhibitory factor which prevents cisplatin from binding transcripts with this feature active. Given this, we should examine the neighborhood around these features and repeat our GSEA to see if we find similar functional enrichment. 

In [5]:
# Pick out the two hop neighborhoods of f/8161 and f/7030
neighbors_8161 = set(nx.single_source_shortest_path_length(NDG, 8161, cutoff=2).keys())
neighbors_7030 = set(nx.single_source_shortest_path_length(NDG, 7030, cutoff=2).keys())

# visualize the neighborhoods
subgraph_8161 = NDG.subgraph(neighbors_8161)
print(subgraph_8161)
subgraph_7030 = NDG.subgraph(neighbors_7030)
print(subgraph_7030)

MultiGraph with 6398 nodes and 667936 edges
MultiGraph with 6035 nodes and 647893 edges


Since the two-hop neighborhood nearly encompassses the entire graph, skip visualization

In [7]:
def feature_geneandmotif_sets(center, edges: list) -> tuple[set, set]:
    genes = []
    motifs = []
    for u, v, annotations in edges:
        motif = u if u != center else v
        if motif:
            motifs.append(motif)        
        for key, val in annotations.items():
            genes.append(val["gene_id"])

    return set(genes), set(motifs)

def GC_content(seq: str) -> float:
    gc_count = sum(1 for base in seq if base in 'GCgc')
    return gc_count / len(seq) if len(seq) > 0 else 0

# pick out immediate neighborhood gene sets
edges_8161 = G.in_edges(8161, data="annotations")
genes_8161, motifs_8161 = feature_geneandmotif_sets(8161, edges_8161)
print("Neighborhood Size of f/8161:", len(motifs_8161), "/", len(genes_8161), "genes")
print("GC content of motifs associated with f/8161:", np.mean([GC_content(motif) for motif in motifs_8161]))
    
edges_7030 = G.in_edges(7030, data="annotations")
genes_7030, motifs_7030 = feature_geneandmotif_sets(7030, edges_7030)
print("Neighborhood Size of f/7030:", len(motifs_7030), "/", len(genes_7030), "genes")
print("GC content of motifs associated with f/7030:", np.mean([GC_content(motif) for motif in motifs_7030]))

Neighborhood Size of f/8161: 4078 / 175 genes
GC content of motifs associated with f/8161: 0.49861860389079615
Neighborhood Size of f/7030: 3728 / 178 genes
GC content of motifs associated with f/7030: 0.48676680972818315


In [9]:
os.makedirs("data/regelem_gene_sets", exist_ok=True)

# export gene sets
with open("data/regelem_gene_sets/genes_f8161.txt", "w") as f:
    for gene in genes_8161:
        f.write(gene + "\n")

with open("data/regelem_gene_sets/genes_f7030.txt", "w") as f:
    for gene in genes_7030:
        f.write(gene + "\n")

# run GSEA in R using clusterProfiler
os.system("Rscript enrich.R --gene_list data/regelem_gene_sets/genes_f8161.txt --outdir data/regelem_gene_sets/enrich_f8161")
os.system("Rscript enrich.R --gene_list data/regelem_gene_sets/genes_f7030.txt --outdir data/regelem_gene_sets/enrich_f7030")


clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

G Yu. Thirteen years of clusterProfiler. The Innovation. 2024,
5(6):100722

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

    filter

enrichplot v1.28.4 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Guang-Rong Yan, Qing-Yu He. DOSE: an
R/Bioconductor package for Disease Ontology Semantic and Enrichment
analysis. Bioinformatics. 2015, 31(4):608-609
Welcome to enrichR
Checking connections ... 
Enrichr ... Connection is Live!
FlyEnrichr ... Connection is Live!
WormEnrichr ... Connection is Live!
YeastEnrichr ... Connection is Live!
FishEnrichr ... Connection is Live!
OxEnrichr ... Connection is Live!

Attaching package: ‘generics’

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, unio

[1] "Reading gene list from: data/regelem_gene_sets/genes_f8161.txt"
[1] "Performing GO enrichment analysis for ontology: BP"
[1] "Generating plots..."
[1] "Saving results to directory: data/regelem_gene_sets/enrich_f8161"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

G Yu. Thirteen years of clusterProfiler. The Innovation. 2024,
5(6):100722

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

  

[1] "Reading gene list from: data/regelem_gene_sets/genes_f7030.txt"
[1] "Performing GO enrichment analysis for ontology: BP"
[1] "Generating plots..."
[1] "Saving results to directory: data/regelem_gene_sets/enrich_f7030"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 


0